# LeNet-5 CIFAR10 Classifier

This notebook implements the classic LeNet-5 convolutional network [1] and applies it to the CIFAR10 object classification dataset. The basic architecture is shown in the figure below:

![](../images/lenet/lenet-5_1.jpg)



LeNet-5 is commonly regarded as the pioneer of convolutional neural networks, consisting of a very simple architecture (by modern standards). In total, LeNet-5 consists of only 7 layers. 3 out of these 7 layers are convolutional layers (C1, C3, C5), which are connected by two average pooling layers (S2 & S4). The penultimate layer is a fully connexted layer (F6), which is followed by the final output layer. The additional details are summarized below:

- All convolutional layers use 5x5 kernels with stride 1.
- The two average pooling (subsampling) layers are 2x2 pixels wide with stride 1.
- Throughrout the network, tanh sigmoid activation functions are used. (**In this notebook, we replace these with ReLU activations**)
- The output layer uses 10 custom Euclidean Radial Basis Function neurons for the output layer. (**In this notebook, we replace these with softmax activations**)

**Please note that the original architecture was applied to MNIST-like grayscale images (1 color channel). CIFAR10 has 3 color-channels. I found that using the regular architecture results in very poor performance on CIFAR10 (approx. 50% ACC). Hence, I am multiplying the number of kernels by a factor of 3 (according to the 3 color channels) in each layer, which improves is a little bit (approx. 60% Acc).**

### References

- [1] Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, november 1998.

## Imports

In [2]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Model Settings

In [3]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 32*32
NUM_CLASSES = 10

# Other
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GRAYSCALE = False

### MNIST Dataset

In [4]:
##########################
### CIFAR-10 Dataset
##########################


# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=8,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Files already downloaded and verified
Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])
Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])


In [5]:
device = torch.device(device)
torch.manual_seed(0)

for epoch in range(2):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


In [6]:
##########################
### MODEL
##########################


class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = nn.Sequential(
            
            nn.Conv2d(in_channels, 6*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6*in_channels, 16*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5*in_channels, 120*in_channels),
            nn.Tanh(),
            nn.Linear(120*in_channels, 84*in_channels),
            nn.Tanh(),
            nn.Linear(84*in_channels, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [7]:
torch.manual_seed(RANDOM_SEED)

model = LeNet5(NUM_CLASSES, GRAYSCALE)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

## Training

In [10]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()

        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

        

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=device)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 0000/0391 | Cost: 1.2905
Epoch: 001/010 | Batch 0050/0391 | Cost: 1.2208
Epoch: 001/010 | Batch 0100/0391 | Cost: 1.3714
Epoch: 001/010 | Batch 0150/0391 | Cost: 1.1746
Epoch: 001/010 | Batch 0200/0391 | Cost: 1.2635
Epoch: 001/010 | Batch 0250/0391 | Cost: 1.1092
Epoch: 001/010 | Batch 0300/0391 | Cost: 1.1314
Epoch: 001/010 | Batch 0350/0391 | Cost: 1.1708
Epoch: 001/010 | Train: 61.238%
Time elapsed: 0.33 min
Epoch: 002/010 | Batch 0000/0391 | Cost: 1.2161
Epoch: 002/010 | Batch 0050/0391 | Cost: 1.1593
Epoch: 002/010 | Batch 0100/0391 | Cost: 1.0635
Epoch: 002/010 | Batch 0150/0391 | Cost: 1.0143
Epoch: 002/010 | Batch 0200/0391 | Cost: 1.1594
Epoch: 002/010 | Batch 0250/0391 | Cost: 1.2889
Epoch: 002/010 | Batch 0300/0391 | Cost: 1.1916
Epoch: 002/010 | Batch 0350/0391 | Cost: 1.0310
Epoch: 002/010 | Train: 64.700%
Time elapsed: 0.65 min
Epoch: 003/010 | Batch 0000/0391 | Cost: 0.9973
Epoch: 003/010 | Batch 0050/0391 | Cost: 1.0985
Epoch: 003/010 | Batch 010

Exception ignored in: <function _releaseLock at 0x7f8502141080>
Traceback (most recent call last):
  File "/home/parsa/anaconda3/envs/mlenv/lib/python3.11/logging/__init__.py", line 237, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


Epoch: 004/010 | Batch 0000/0391 | Cost: 0.8861
Epoch: 004/010 | Batch 0050/0391 | Cost: 0.9822
Epoch: 004/010 | Batch 0100/0391 | Cost: 0.7781
Epoch: 004/010 | Batch 0150/0391 | Cost: 0.9054
Epoch: 004/010 | Batch 0200/0391 | Cost: 0.9229
Epoch: 004/010 | Batch 0250/0391 | Cost: 0.8809
Epoch: 004/010 | Batch 0300/0391 | Cost: 1.0467
Epoch: 004/010 | Batch 0350/0391 | Cost: 0.7888
Epoch: 004/010 | Train: 71.884%
Time elapsed: 1.60 min
Epoch: 005/010 | Batch 0000/0391 | Cost: 0.7545
Epoch: 005/010 | Batch 0050/0391 | Cost: 0.8506
Epoch: 005/010 | Batch 0100/0391 | Cost: 0.8146
Epoch: 005/010 | Batch 0150/0391 | Cost: 0.8738
Epoch: 005/010 | Batch 0200/0391 | Cost: 0.8487


## Evaluation

In [9]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 67.30%


In [10]:
%watermark -iv

torch       1.3.1
pandas      0.24.2
PIL.Image   6.2.1
torchvision 0.4.2
matplotlib  3.1.0
numpy       1.17.4

